In [ ]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

In [ ]:
x_train = np.load('./data/mnist/x_train.npy')
x_test = np.load('./data/mnist/x_test.npy')
y_train_cls = np.load('./data/mnist/y_train.npy')
y_test_cls = np.load('./data/mnist/y_test.npy')

In [ ]:
num_pixels = x_train.shape[1]
num_channels = 1

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
y_train = np_utils.to_categorical(y_train_cls)
y_test = np_utils.to_categorical(y_test_cls)
num_classes = y_train[0].size

In [ ]:
filter_size1 = 5
num_filters1 = 16

filter_size2 = 5
num_filters2 = 36

fc_size = 128

In [ ]:
x = tf.placeholder(tf.float32, [None, num_pixels, num_pixels])
x_img = tf.reshape(x,[-1,num_pixels,num_pixels,num_channels])

In [ ]:
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

In [ ]:
step = tf.placeholder(tf.int32)

In [ ]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [ ]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [ ]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    
    layer = tf.nn.conv2d(input = input, filter = weights, strides = [1,1,1,1], padding='SAME')
    layer += biases
    
    if use_pooling:
        layer = tf.nn.max_pool(value=layer, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    layer = tf.nn.relu(layer)
    print("No. of parameters: " +  str(layer.get_shape()[1:4].num_elements()))
    return layer, weights

In [ ]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1,num_features])
    return layer_flat, num_features

In [ ]:
def new_fc_layer(input,num_inputs, num_outputs, use_relu=True):
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    layer=tf.matmul(input,weights)+biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [ ]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_img, num_input_channels=num_channels, filter_size=filter_size1, num_filters=num_filters1, use_pooling=True)

In [ ]:
layer_conv1

In [ ]:
layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1, num_input_channels=num_filters1, filter_size=filter_size2, num_filters=num_filters2, use_pooling=True)

In [ ]:
layer_conv2

In [ ]:
layer_flat, num_features = flatten_layer(layer_conv2)

In [ ]:
layer_flat

In [ ]:
layer_fc1 = new_fc_layer(input=layer_flat, num_inputs=num_features, num_outputs=fc_size, use_relu=True)

In [ ]:
layer_fc2 = new_fc_layer(input=layer_fc1, num_inputs=fc_size, num_outputs=num_classes, use_relu=False)

In [ ]:
layer_fc2

In [ ]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

In [ ]:
cost = tf.reduce_mean(cross_entropy)

In [ ]:
lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/np.e)
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

In [ ]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
session=tf.Session()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
batch_size = 64

In [ ]:
def print_accuracy():
    acc = session.run(accuracy, feed_dict = feed_dict_test)
    print("Accuracy on test-set:{0:.1%}".format(acc))

In [ ]:
def optimize(num_iterations):
    for j in range(num_iterations):
        print("Iteration: " + str(j))
        for i in range(batch_size, x_train.shape[0], batch_size):
            x_batch = x_train[i-batch_size:i]
            y_true_batch = y_train[i-batch_size:i]
            feed_dict_train = {x: x_batch, y_true: y_true_batch, step: j}
            session.run(optimizer, feed_dict=feed_dict_train)
        print_accuracy()

In [ ]:
feed_dict_test = {x:x_test, y_true: y_test, y_true_cls: y_test_cls}

In [ ]:
print_accuracy()

In [ ]:
weights_conv1.eval(session=session)

In [ ]:
weights_conv2.eval(session=session)

In [ ]:
optimize(num_iterations=2)

In [ ]:
weights_conv1.eval(session=session)

In [ ]:
weights_conv2.eval(session=session)